In [6]:
import requests
import os
import pandas as pd
tokenCanvas = os.environ['tokenCanvas']

headers = {
    'Authorization': f'Bearer {tokenCanvas}',
}
params = {
    'per_page': 100,
    'sort': 'last_login',
}

def finn_rel(link_header):
    link_header_dict = {}
    for link in link_header.split(","):
        url, rel = link.strip().split(";")
        rel = rel.split('=')[1]
        link_header_dict[rel.strip().replace('"', '')] = url.strip().replace('<', '').replace('>', '')      # Kommentar: denne er med BERRE for å skape balanse i .org-fila: \"
    return link_header_dict

url = "https://hvl.instructure.com/api/v1/accounts/1/users"

# Hente siste aktivitet
Dette kan vere ein alternativ måte å hente siste aktivitet på: hente alle brukarar via `/api/v1/accounts/1/users` med parameter `'sort': 'last_login'`. Men eg henta dei jo i februar, så eg ser på den først.

In [ ]:
dr_liste = []
hentmeir = True
while hentmeir:
    respons = requests.get(url, headers=headers, params=params)
    if 200 <= respons.status_code < 300:
        data = respons.json()
        hentmeir = "next" in respons.headers['link']
        if hentmeir:
            url = finn_rel(respons.headers['link'])['next']
            print(url)   # for kontrollen sin skuld
        dataliste = []
        for element in data:
        # ta med alle
            dataliste.append(element)
        # eller ta ut enkelte kolonner
        #     dataliste.append([element['id'], element['name'], element['enrollment_term_id']])
        df = pd.DataFrame(dataliste, columns=['id', 'name', 'enrollment_term_id'])
        dr_liste.append(df)
# og etter at eg er ferdig å lese inn slår eg dei same:
alledata = pd.concat((df for df in dr_liste if not df.empty), ignore_index=True)
